CANapeAPI Python Module.

In [2]:
!rm -rf tmp_*_canape

In [1]:
import ctypes

In [3]:
from enum import IntEnum

In [42]:
class TAsap3ECUState(ctypes.c_uint):
    def __init__(self, value):
        self.states = {
            0: "Online",
            1: "Offline"
        }
        assert value in self.states,  Exception("Unknown State")
        self.value=value
    
    def __eq__(self):
        raise Exception(U)
        
    @property
    def state(self):
        if self.value in self.states:
            return self.states[self.value]
        
    def __repr__(self):
        return f"ECUState<{self.state}>" 

In [51]:
def generate_fancy_enum_factory(states, name):
    class FancyEnum(ctypes.c_uint):
        def __init__(self, value):
            self.states = states
            assert value in self.states,  Exception("Unknown State")
            self.value=value

        def __eq__(self):
            raise Exception(U)

        @property
        def state(self):
            if self.value in self.states:
                return self.states[self.value]

        def __repr__(self):
            return f"{name}<{self.state}>" 
    return FancyEnum

In [55]:
TAsap3ECUState=generate_fancy_enum_factory(
    {
        0: "Online",
        1: "Offline"
    },
    "ECUState"    
)

In [56]:
TAsap3ECUState(0)

ECUState<Online>

In [57]:
TAsap3ECUState(1)

ECUState<Offline>

In [59]:
TAsap3ECUState(2)

AssertionError: Unknown State

In [60]:
TAsap3ECUState=generate_fancy_enum_factory(
    {
        0: "Online",
        1: "Offline"
    },
    "ECUState"    
)

In [1]:
import CANapAPI
import os
import platform
import ctypes
import tempfile
working_directory=os.path.abspath(tempfile.mkdtemp(suffix="_canape",prefix="tmp_",dir=os.curdir))
class MeasurementState(object):
    def __init__(self, state):
        self.state=state
    def __repr__(self):
        return "<MeasurementState({},{})>".format(self.state,self.__str__())
    def __str__(self):
        if self.state==0:
            return "Stopped"
        elif self.state==1:
            return "Init"
        elif self.state==2:
            return "Stop On Start"
        elif self.state==3:
            return "Exit"
        elif self.state==4:
            return "Thread Running"
        elif self.state==5:
            return "Running"
        else:
            return "ERROR"

SyntaxError: invalid syntax (CANapAPI.py, line 379)

In [ ]:
cty

In [96]:
try:
    tmp.SetInteractiveMode(False)
    tmp.Exit()
except:
    pass
class CANape(object):
    # Known default install paths. In descending order of newest to oldest.
    paths=[r"C:\Program Files\Vector CANape 17",
           r"C:\Program Files (x86)\Vector CANape 14",
           r"C:\Program Files (x86)\Vector CANape 13",
           r"C:\Program Files (x86)\Vector\CANape\12",
           r"C:\Program Files (x86)\Vector\CANape\11"]
    # Defaults
    responseTimeout = 10000 # 10 seconds
    fifoSize = 8192
    sampleSize = 256
    debugMode = True
    clearDeviceList = True
    bHexmode = False
    bModalMode = False
        
    def __init__(self, canape_path=None, api_path=None, dll=None):
        # If the DLL path isn't explicitly given.        
        if dll is None:
            # If an API path isn't explicitly given.
            if api_path is None:
                # Loop through each of the known CANape paths.
                for path in CANape.paths:
                    # If the path exists.
                    if os.path.exists(path):
                        # Set the CANape
                        canape_path=path
                        # Break the loop.
                        break
                # If the CANape path is still none.
                if canape_path is None:
                    # Raise an exception that CANape wasn't found.
                    raise Exception("CANape directory not found")
                # Set the API path based on the CANape path.
                self.api_path = os.path.join(canape_path,"CANapeAPI") 
            else:
                self.api_path = api_path
            # Load the DLL based on the version of Python
            if platform.architecture()[0] == '32bit':
                self.dll = os.path.join(self.api_path,"CANapAPI.dll")
            else:
                self.dll = os.path.join(self.api_path,"CANapAPI64.dll")
        else:
            self.dll = dll
        # Load the DLL.
        self.api = ctypes.WinDLL(self.dll)
        
        #  Version control
        #  ~~~~~~~~~~~~~~~  
        #  Should be executed, the received data should be compared.
        #   The current DLL is suitable,
        #   if(version.dllMainVersion == CANAPE_API_MAIN_VESION) and
        #      (version.dllSubVersion  == CANAPE_API_SUB_VESION) and
        #      (version.dllRelease     >=  CANAPE_API_RELEASE)

        self.ver = CANapAPI.version_t()
        self.api.Asap3GetVersion(ctypes.byref(self.ver))
#        if not((version.dllMainVersion == CANapAPI.CANAPE_API_MAIN_VESION) and \
#            (version.dllSubVersion == CANapAPI.CANAPE_API_SUB_VESION) and \
#            (version.dllRelease >= CANapAPI.CANAPE_API_RELEASE)):
#            raise Exception("DLL API version does not match py generated from .h file API Version")
        
        self.modules = list()
        self.handle = CANapAPI.TAsap3Hdl()
        
    #  Trouble shooting: call this function to popup the debug window of the MCD-system.
    #   hdl [input]: Asap3 Handle
    #   fileName [input]: Content of the debug window is saved to this file.
    def SaveDebugWindow(self, fileName):
        return self.api.Asap3SaveDebugWindow(self.handle, fileName)

    #  Returns the count of instantiated Modules in the current Project
    #   hdl [input]: Asap3 Handle
    #   count [output]:  returns the count of modules
    def GetModuleCount(self):
        count = ctypes.c_ulong()
        self.api.Asap3GetModuleCount(self.handle, count)
        return count

    #  Get name of attached ASAP2 file
    #   hdl [input]: Asap3 Handle
    #   module [input]: Points to demanded module
    #   asap2Fname [output]: Contains data description file name after call
    def GetAsap2(self, module=0):
        module = CANapAPI.TModulHdl(module)
        asap2Fname = ctypes.c_char_p()
        self.api.Asap3GetAsap2(self.handle, module, ctypes.byref(asap2Fname))
        return asap2Fname.value

    def running(self):
        # Determine if CANape is running.
        try:
            # Check the handle contents.
            tmp.handle.contents
            # If it makes it this far, it's not a null pointer.
            return True
        except ValueError:
            # If a ValueError is raised (null pointer), CANape isn't running.
            return False
        except:
            raise
    def dump(self):
        # Quick and dirty way to print attributes.
        for var in vars(self):
            print("{}: {}".format(var,getattr(self,var)))
        
    def GetVersion(self):
        ver = CANapAPI.version_t()
        self.api.Asap3GetVersion(ctypes.byref(ver))
        return ver
    
    @property
    def version(self):
        ver = self.GetVersion()
        return "{}.{}.{}".format(ver.dllMainVersion, ver.dllSubVersion, ver.dllRelease)
     
    # Initialize ASAP3 connection
    # Returns a handle to be used in subsequent function calls. 
    # CANape will be started (if not actually running).
    def Init(self,
             workingDir=None,
             responseTimeout=None,
             fifoSize=None,
             debugMode=None):
        if workingDir is None:
            workingDir = os.path.abspath(os.path.curdir)
        if responseTimeout is None:
            responseTimeout = CANape.responseTimeout
        if fifoSize is None:
            fifoSize = CANape.fifoSize
        if debugMode is None:
            debugMode = CANape.debugMode
        return self.api.Asap3Init(ctypes.byref(self.handle),
                     responseTimeout,
                     workingDir,
                     fifoSize,
                     debugMode)
    def Init2(self,
             workingDir=None,
             responseTimeout=None,
             fifoSize=None,
             sampleSize=None,
             debugMode=None):
        if workingDir is None:
            workingDir = os.path.abspath(os.path.curdir)
        if responseTimeout is None:
            responseTimeout = CANape.responseTimeout
        if fifoSize is None:
            fifoSize = CANape.fifoSize
        if sampleSize is None:
            sampleSize = CANape.sampleSize
        if debugMode is None:
            debugMode = CANape.debugMode
        return self.api.Asap3Init2(ctypes.byref(self.handle),
                              responseTimeout,
                              workingDir,
                              fifoSize,
                              sampleSize,
                              debugMode)

    def Init3(self,
             workingDir=None,
             responseTimeout=None,
             fifoSize=None,
             sampleSize=None,
             debugMode=None, 
             clearDeviceList=None):
        #  In addition to Asap3Init2() the device list of CANape is only cleared if the value of clearDeviceList is 'true'.
        #   hdl [output]: Asap3Handle for further access to this interface.
        #   responseTimeout [input]: Maximum response time
        #   workingDir [input]: Sets CANape working directory
        #         By default the project file CANape.INI saved at the working directory
        #         is used for the CANape session. If you want to load a different
        #         project file. please append the project file name to 'workingDir'.
        #   fifoSize [input]: Total size of FIFO used for data acquisition, i.e. number of FIFO
        #         entries which can be read out using Asap3GetNextSample(). Each FIFO entry
        #         includes at most ACQ_MAX_VALUES=128 measurement values.
        #   sampleSize [input]: The maximum number of measurement values per FIFO entry is 256.
        #   debugMode [input]: if this is true call CANape in 'normal' screen size instead of 'minimized'.
        #           if it is false call CANape in 'minimized' mode.
        # clearDeviceList [input]: If it is true the CANape device list will be cleared.
        if workingDir is None:
            workingDir = os.path.abspath(os.path.curdir)
        if responseTimeout is None:
            responseTimeout = CANape.responseTimeout
        if fifoSize is None:
            fifoSize = CANape.fifoSize
        if sampleSize is None:
            sampleSize = CANape.sampleSize
        if debugMode is None:
            debugMode = CANape.debugMode
        if clearDeviceList is None:
            clearDeviceList = CANape.clearDeviceList
        return self.api.Asap3Init3(ctypes.byref(self.handle),
                      responseTimeout,
                      workingDir,
                      fifoSize,
                      sampleSize,
                      debugMode,
                      clearDeviceList)

    def Init4(self,
             workingDir=None,
             responseTimeout=None,
             fifoSize=None,
             sampleSize=None,
             debugMode=None, 
             clearDeviceList=None,
             bHexmode=None):
        #  In addition to Asap3Init3() CANape is started in Hexmode if value of bHexmode is 'true'.
        #  Hex is a special CANape mode to view your databases or/and hexfiles without an device.
        #   In this mode data acqusition is impossible.
        #   hdl [output]: Asap3Handle for further access to this interface.
        #   responseTimeout [input]: Maximum response time
        #   workingDir [input]: Sets CANape working directory
        #         By default the project file CANape.INI saved at the working directory
        #         is used for the CANape session. If you want to load a different
        #         project file, please append the project file name to 'workingDir'.
        #   fifoSize [input]: Total size of FIFO used for data acquisition, i.e. number of FIFO
        #         entries which can be read out using Asap3GetNextSample(). Each FIFO entry
        #         includes at most ACQ_MAX_VALUES=128 measurement values.
        #   sampleSize [input]: The maximum number of measurement values per FIFO entry is 256.
        #   debugMode [input]: if this is true call CANape in 'normal' screen size instead of 'minimized'.
        #           if it is false call CANape in 'minimized' mode.
        #   clearDeviceList [input]: If it is true the CANape device list will be cleared.
        #   bHexmode [input]: If it is true the CANape will be started in HexMode
        if workingDir is None:
            workingDir = os.path.abspath(os.path.curdir)
        if responseTimeout is None:
            responseTimeout = CANape.responseTimeout
        if fifoSize is None:
            fifoSize = CANape.fifoSize
        if sampleSize is None:
            sampleSize = CANape.sampleSize
        if debugMode is None:
            debugMode = CANape.debugMode
        if clearDeviceList is None:
            clearDeviceList = CANape.clearDeviceList
        if bHexmode is None:
            bHexmode = CANape.bHexmode
        return self.api.Asap3Init4(ctypes.byref(self.handle),
                      responseTimeout,
                      workingDir,
                      fifoSize,
                      sampleSize,
                      debugMode,
                      clearDeviceList,
                      bHexmode)
    
    def Init5(self,
             workingDir=None,
             responseTimeout=None,
             fifoSize=None,
             sampleSize=None,
             debugMode=None, 
             clearDeviceList=None,
             bHexmode=None,
             bModalMode=None):
        #  In addition to Asap3Init4() CANape is started in nonmodal mode, if value of bModalMode is 'false'.
        #   hdl [output]: Asap3Handle for further access to this interface.
        #   responseTimeout [input]: Maximum response time
        #   workingDir [input]: Sets CANape  working directory
        #         By default the project file CANape.INI saved at the working directory
        #         is used for the CANape session. If you want to load a different
        #         project file, please append the project file name to 'workingDir'.
        #   fifoSize [input]: Total size of FIFO used for data acquisition, i.e. number of FIFO
        #         entries which can be read out using Asap3GetNextSample(). Each FIFO entry
        #         includes at most ACQ_MAX_VALUES=128 measurement values.
        #   sampleSize [input]: The maximum number of measurement values per FIFO entry is 256.
        #   debugMode [input]: if this is true call CANape in 'normal' screen size instead of 'minimized'.
        #           if it is false call CANape in 'minimized' mode.
        #   clearDeviceList [input]: If it is true the CANape device list will be cleared.
        #   bHexmode [input]: If it is true the CANape will be started in HexMode
        #   bModalMode [input]: If it is true the CANape will be started in the NONMODAL mode.
        if workingDir is None:
            workingDir = os.path.abspath(os.path.curdir)
        if responseTimeout is None:
            responseTimeout = CANape.responseTimeout
        if fifoSize is None:
            fifoSize = CANape.fifoSize
        if sampleSize is None:
            sampleSize = CANape.sampleSize
        if debugMode is None:
            debugMode = CANape.debugMode
        if clearDeviceList is None:
            clearDeviceList = CANape.clearDeviceList
        if bHexmode is None:
            bHexmode = CANape.bHexmode
        if bModalMode is None:
            bModalMode = CANape.bModalMode
        self.api.Asap3Init5(ctypes.byref(self.handle),
                              responseTimeout,
                              workingDir,
                              fifoSize,
                              sampleSize,
                              debugMode,
                              clearDeviceList,
                              bHexmode,
                              bModalMode);
    def Exit(self):
        return self.api.Asap3Exit(self.handle);
    
    #  Shut down ASAP3 connection to CANape with optional termination of CANape. true
    #   hdl [input]: Asap3 Handle
    #   close_CANape [input]: true -> CANape will be shutdown otherwise CANape will not be shutdown
    def Exit2(self, close_CANape = True):
        return self.api.Asap3Exit2(self.handle, close_CANape);
    
    #  Trouble shooting: call this function to popup the debug window of the MCD-system.
    #   hdl [input]: Asap3 Handle
    def PopupDebugWindow(self):
        return self.api.Asap3PopupDebugWindow(self.handle)
    
    #  Call this function to get the current version of the server application(CANape).
    #   hdl [input]: Asap3 Handle
    #   version [output]: version info
    def GetApplicationVersion(self):
        version = CANapAPI.Appversion()
        self.api.Asap3GetApplicationVersion(self.handle, ctypes.byref(version))
        return version
    
    #  Asks CANape wheather a ECU is online or offline
    #   hdl [input]: Asap3 Handle
    #   module [input]: Points to demanded module
    #   State [output]: pointer on a stateflag( online - offline )
    def IsECUOnline(self, module):
        State = CANapAPI.TAsap3ECUState()
        self.api.Asap3IsECUOnline(self.handle, module, ctypes.byref(State))
        return State
    
    #  Create a new module/device and attach an ASAP2-description file
    #  Returns a handle to be used for subsequent function calls
    #   hdl [input]: Asap3 Handle
    #   asap2Fname [input]: Name of the asap2 file to load.
    #   canChnl [input]:   CAN channel to select.
    #   module [output]:  After call, it contains handle to new created module.
    def AttachAsap2(self, a2l, canChnl=1):
        module=CANapAPI.TModulHdl
        self.api.Asap3AttachAsap2(self.handle, a2l, canChnl, ctypes.byref(module))
        self.modules.append(module)
        
    
    #  Returns the count of instantiated Modules in the current Project
    #   hdl [input]: Asap3 Handle
    #   count [output]:  returns the count of modules
    def GetModuleCount(self):
        count = ctypes.c_long()
        self.api.Asap3GetModuleCount(self.handle, ctypes.byref(count))
        return count
    
    #  Create a new module/device and attach an description file(A2L, DB, DBC)
    #   Returns a handle to be used for subsequent function calls see: tDriverType
    #   hdl [input]: Asap3 Handle
    #   moduleName [input]: Name of module to create.
    #   databaseFilename [input]: Name of description file to load
    #   driverType [input]: Set driver type. Possible values see tDriverType
    #   channelNo [input]: Logical communication channel to be used(like CCP:1-4 = CAN1-CAN4, 255 = TCP/IP, 256 = UDP))
    #   module [output]: After call, it contains handle to new created module.
    def CreateModule(self, moduleName, databaseFilename, driverType, channelNo):
        module=CANapAPI.TModulHdl()
        self.api.Asap3CreateModule(self.handle,
                                   moduleName,
                                   databaseFilename,
                                   driverType,
                                   channelNo,
                                   ctypes.byref(module))
        self.modules.append(module)
        
    #   Enables the Interactive mode of CANape.
    #   hdl [input]: Asap3 handle
    #   mode [input]: Set this parameter to true to enable the interactive mode, otherwise set this paramater to false
    def SetInteractiveMode(self, mode):
        return self.api.Asap3SetInteractiveMode(self.handle, mode)

    #   Returns information about the InteractiveMode.
    #   hdl [input]: Asap3 handle
    #   mode [input]: This parameter returns true if the interactive mode is enabled, otherwise false will be returned
    def GetInteractiveMode(self):
        mode = ctypes.c_bool(0)
        self.api.Asap3GetInteractiveMode(self.handle, ctypes.byref(mode))
        return mode
    
    #  returns the current state of the measurement
    #   hdl [input]: Asap3 Handle
    #   module [input]: Points to demanded module.
    #   State [output]: pointer to recieve the measuremnt state
    def GetMeasurementState(self):
        State=CANapAPI.tMeasurementState()
        self.api.Asap3GetMeasurementState(self.handle, ctypes.byref(State))
        return MeasurementState(State.value)
        
    #  Switches an ECU from online to offline and vice versa
    #   hdl [input]: Asap3 Handle
    #   module [input]: Points to demanded module
    #   State [input]: stateflag( online - offline )
    #   download [input]: if this parameter is set to true CANape will execute an download in case of
    #                          TAsap3ECUState = TYPE_SWITCH_ONLINE
    def ECUOnOffline(self, module, State, download):
        return self.api.Asap3ECUOnOffline(self.handle, module, State, download)
tmp=CANape()
# Launch in Modal mode (So we can interact with it)
tmp.Init5(workingDir=working_directory, bModalMode=True)
a2l=r"C:\GIT_REPOS\lmt_mach_sys_bench_setups\test_measurement_setups\CANape\Chassis\Tier4_Chassis_785G_std.a2l"
tmp.CreateModule(moduleName="Tier4_Chassis_785G_std",
                databaseFilename=a2l,
                driverType=CANapAPI.ASAP3_DRIVER_CCP,
                channelNo=1)